<h2>Creating Table for Ranking<h2>

In [1]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder.appName("Ranking Features")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/09 11:14:42 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.16 instead (on interface en0)
22/10/09 11:14:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 11:14:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/09 11:14:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
transaction_data = spark.read.parquet('../data/curated/cleaned_data.parquet')
merchant_fraud_rates = spark.read.csv('../data/curated/merchant_fraud_rate.csv', header=True)
consumer_fraud_rates = spark.read.csv('../data/curated/consumer_fraud_rate.csv', header=True)
future_profit = spark.read.csv('../data/curated/future_predictions.csv', header=True)

In [13]:
transaction_data = transaction_data.join(consumer_fraud_rates, ['user_id'])

In [14]:
merchant_properties = transaction_data.groupBy("merchant_abn").agg(\
    F.avg('Median_age_persons').alias("avg_expected_customer_age"),\
    F.avg('fraud_rate').alias("avg_customer_fraud_rate")\
)

In [15]:
future_profit = future_profit.drop("_c0")

In [16]:
merchant_ranking_properties = merchant_properties.join(future_profit, ['merchant_abn']).join(merchant_fraud_rates, ['merchant_abn'])

In [17]:
merchant_ranking_properties.write.mode('overwrite').option('header', True).csv('../data/curated/merchant_ranking_properties.csv')

22/10/09 16:23:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 946657 ms exceeds timeout 120000 ms
22/10/09 16:23:02 WARN SparkContext: Killing executors is not supported by current scheduler.
